# TF_IDF_Word_Embedding

## TF - IDF Vectorizer


In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# Konfigurasi Pandas untuk tampilan penuh
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

# Fungsi untuk membersihkan teks
def clean_text(text):
    """
    Membersihkan teks dengan mengubahnya menjadi huruf kecil,
    menghapus tanda baca, angka, dan spasi berlebih.
    """
    if not isinstance(text, str):
        return ''
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Menggunakan try-except untuk membaca data dari file CSV
try:
    # Membaca seluruh data dari file CSV
    df = pd.read_csv('cleaned_detik_berita_5_pages.csv')
    print("Berhasil memuat seluruh data dari file CSV.")
    # Gunakan 'isi_berita_diproses' sebagai korpus
    corpus = df['isi_berita_diproses'].tolist()
except FileNotFoundError:
    print("Error: File 'cleaned_detik_berita_5_pages.csv' tidak ditemukan.")
    print("Menggunakan data dummy untuk demonstrasi.")
    corpus = [
        "Pelatih Atletico Madrid, Diego Simeone, mengaku kesal saat dihina fans Liverpool sepanjang pertandingan.",
        "Atletico Madrid takluk dari Liverpool dengan skor 2-3 di Anfield pada lanjutan Liga Champions.",
        "Bicara soal seberapa hebat Lionel Messi, Zinedine Zidane punya sudut pandang berbeda.",
        "Marc Marquez menjalani operasi pada lengannya usai kecelakaan di MotoGP Portugal 2025.",
        "Pebalap Yamaha, Alex Rins akan menggunakan motor M1 yang dikembangkan untuk 2026."
    ]

# Menerapkan pembersihan teks pada korpus
cleaned_corpus = [clean_text(text) for text in corpus]

# Inisialisasi CountVectorizer untuk menghitung Term Frequency (TF)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(cleaned_corpus)
count_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

# Inisialisasi TfidfTransformer untuk menghitung IDF dan TF-IDF
transformer = TfidfTransformer()

# ---
### 1. Hasil Count Vectorizer (Matriks Term Frequency)

print("---")
print(">>> Count Vectorizer (Matriks Term Frequency):")
print(count_df.to_string())
print("-" * 50)

# ---
### 2. Hasil IDF (Inverse Document Frequency)

transformer.fit(X)
idf_values = transformer.idf_
feature_names = vectorizer.get_feature_names_out()
idf_df = pd.DataFrame({'word': feature_names, 'idf_value': idf_values})
print(">>> IDF (Inverse Document Frequency):")
print(idf_df.to_string())
print("-" * 50)

# ---
### 3. Hasil TF-IDF (Term Frequency - Inverse Document Frequency)

tfidf_matrix = transformer.transform(X)
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)
print(">>> TF-IDF (Term Frequency - Inverse Document Frequency):")
print(tfidf_df.to_string())
print("-" * 50)

Error: File 'cleaned_detik_berita_5_pages.csv' tidak ditemukan.
Menggunakan data dummy untuk demonstrasi.
---
>>> Count Vectorizer (Matriks Term Frequency):
   akan  alex  anfield  atletico  berbeda  bicara  champions  dari  dengan  di  diego  dihina  dikembangkan  fans  hebat  kecelakaan  kesal  lanjutan  lengannya  liga  lionel  liverpool  madrid  marc  marquez  mengaku  menggunakan  menjalani  messi  motogp  motor  operasi  pada  pandang  pebalap  pelatih  pertandingan  portugal  punya  rins  saat  seberapa  sepanjang  simeone  skor  soal  sudut  takluk  untuk  usai  yamaha  yang  zidane  zinedine
0     0     0        0         1        0       0          0     0       0   0      1       1             0     1      0           0      1         0          0     0       0          1       1     0        0        1            0          0      0       0      0        0     0        0        0        1             1         0      0     0     1         0          1        1     0     0  

## Word Embeding

In [ ]:
# Perintah instalasi pip harus diawali dengan tanda seru (!)
!pip install pandas numpy gensim spacy transformers torch

# Perintah untuk mengunduh model spaCy juga harus diawali dengan tanda seru (!)
!python -m spacy download en_core_web_sm

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import pandas as pd
import re
import numpy as np
from gensim.models import Word2Vec
import spacy
from transformers import pipeline

# ---
# Bagian 1: Word Embedding (Pembuatan Representasi Vektor) dengan Gensim
# ---

# Konfigurasi Pandas dan Numpy untuk tampilan penuh
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
np.set_printoptions(threshold=np.inf)

print(">>> Bagian 1: Membuat Word Embedding dengan Gensim")
print("---")

# Membaca seluruh data dari file CSV
try:
    df = pd.read_csv('cleaned_detik_berita_5_pages.csv')
    print("Berhasil memuat seluruh data dari file CSV.")
except FileNotFoundError:
    print("Error: File 'cleaned_detik_berita_5_pages.csv' tidak ditemukan.")
    print("Membuat data dummy untuk demonstrasi.")
    data = {'isi_berita_diproses': ["presiden jokowi bertemu dengan para menteri",
                                    "indonesia berhasil meraih medali emas di bulu tangkis",
                                    "jakarta diguyur hujan deras hari ini"]}
    df = pd.DataFrame(data)

# Gunakan 'isi_berita_diproses' sebagai korpus
corpus = df['isi_berita_diproses'].tolist()

# Fungsi pembersihan teks
def clean_text(text):
    if not isinstance(text, str):
        return ''
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Menerapkan pembersihan teks dan tokenisasi ke seluruh korpus
# Gensim membutuhkan daftar daftar kata (list of lists of words)
tokenized_corpus = [clean_text(text).split() for text in corpus]

# Membangun dan melatih model Word2Vec
# size = dimensi vektor, window = ukuran jendela, min_count = frekuensi kata minimum
embedding_dim = 128
if tokenized_corpus and any(tokenized_corpus):
    word2vec_model = Word2Vec(sentences=tokenized_corpus,
                              vector_size=embedding_dim,
                              window=5,
                              min_count=1,
                              sg=0)  # sg=0 untuk CBOW, sg=1 untuk Skip-gram
    word2vec_model.train(tokenized_corpus, total_examples=word2vec_model.corpus_count, epochs=10)

    # Mengambil matriks embedding dan daftar kata (vocabulary)
    words = list(word2vec_model.wv.index_to_key)
    vectors = [word2vec_model.wv[word] for word in words]

    # Membuat DataFrame untuk tampilan
    embedding_df = pd.DataFrame(vectors, index=words)

    print("\n>>> Word Embeddings yang sudah terlatih (Seluruh Matriks):")
    print(embedding_df.to_string())
else:
    print("\nKorpus kosong atau tidak valid, tidak dapat membuat embeddings.")

print("-" * 50)

# ---
# Bagian 2: Analisis Sentimen (Menggunakan Model Bahasa Indonesia)
# ---

print("\n>>> Bagian 2: Analisis Sentimen (Menggunakan Model Bahasa Indonesia)")
print("---")

try:
    # Menggunakan model Hugging Face untuk analisis sentimen Bahasa Indonesia
    classifier = pipeline("sentiment-analysis", model="w11wo/indonesian-roberta-base-sentiment-classifier")

    ulasan_positif = "Produk ini sangat bagus, saya benar-benar menyukainya!"
    ulasan_negatif = "Layanan sangat mengecewakan, saya tidak akan kembali."
    ulasan_netral = "Barang datang tepat waktu dan sesuai deskripsi."

    hasil_positif = classifier(ulasan_positif)[0]
    hasil_negatif = classifier(ulasan_negatif)[0]
    hasil_netral = classifier(ulasan_netral)[0]

    print(f"Ulasan 1: '{ulasan_positif}'")
    print(f"    Label: {hasil_positif['label']} -> Skor: {hasil_positif['score']:.4f}")
    print("\n" + "-"*30)
    print(f"Ulasan 2: '{ulasan_negatif}'")
    print(f"    Label: {hasil_negatif['label']} -> Skor: {hasil_negatif['score']:.4f}")
    print("\n" + "-"*30)
    print(f"Ulasan 3: '{ulasan_netral}'")
    print(f"    Label: {hasil_netral['label']} -> Skor: {hasil_netral['score']:.4f}")
    print("-" * 50)

except Exception as e:
    print(f"Terjadi kesalahan saat menjalankan analisis sentimen: {e}")
    print("Pastikan Anda telah menginstal pustaka 'transformers' dan 'torch'.")
    print("-" * 50)

# ---
# Bagian 3: Deteksi Entitas Bernama (Menggunakan spaCy)
# ---

print("\n>>> Bagian 3: Deteksi Entitas Bernama (Menggunakan spaCy)")
print("---")

try:
    # Memuat model spaCy untuk bahasa Inggris
    nlp = spacy.load("en_core_web_sm")
    teks_ner = "Apple Inc. founded by Steve Jobs in California, is a major technology company."

    doc = nlp(teks_ner)

    print(f"Teks Asli: {teks_ner}")
    print("\nEntitas yang Terdeteksi:")
    for ent in doc.ents:
        print(f"    - Teks: {ent.text}, Label: {ent.label_}")
    print("-" * 50)

except OSError:
    print("Model spaCy 'en_core_web_sm' tidak ditemukan. Silakan instal dengan perintah:")
    print("python -m spacy download en_core_web_sm")
    print("-" * 50)

>>> Bagian 1: Membuat Word Embedding dengan Gensim
---
Error: File 'cleaned_detik_berita_5_pages.csv' tidak ditemukan.
Membuat data dummy untuk demonstrasi.

>>> Word Embeddings yang sudah terlatih (Seluruh Matriks):
                0         1         2         3         4         5         6         7         8         9         10        11        12        13            14        15        16        17        18        19        20        21        22        23        24        25        26        27        28        29        30        31        32        33        34        35        36        37        38        39        40        41        42        43        44        45        46        47        48        49        50        51        52        53        54        55        56        57        58        59        60        61        62        63        64        65        66        67        68        69        70        71        72        73        74        75        76 

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/328 [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu


Ulasan 1: 'Produk ini sangat bagus, saya benar-benar menyukainya!'
    Label: positive -> Skor: 0.9980

------------------------------
Ulasan 2: 'Layanan sangat mengecewakan, saya tidak akan kembali.'
    Label: negative -> Skor: 0.9993

------------------------------
Ulasan 3: 'Barang datang tepat waktu dan sesuai deskripsi.'
    Label: positive -> Skor: 0.9537
--------------------------------------------------

>>> Bagian 3: Deteksi Entitas Bernama (Menggunakan spaCy)
---
Teks Asli: Apple Inc. founded by Steve Jobs in California, is a major technology company.

Entitas yang Terdeteksi:
    - Teks: Apple Inc., Label: ORG
    - Teks: Steve Jobs, Label: PERSON
    - Teks: California, Label: GPE
--------------------------------------------------


## Klasifikasi

In [ ]:
!pip install pandas scikit-learn tensorflow numpy

In [ ]:
import os
print(os.getcwd())
print(os.listdir())

/content
['.config', 'sample_data']


In [ ]:
from google.colab import files
files.upload()  # lalu pilih file CSV dari komputer

Saving detik_berita_10_pages.csv to detik_berita_10_pages.csv


{'detik_berita_10_pages.csv': b'\xef\xbb\xbfid_berita,judul_berita,isi_berita_original,isi_berita_diproses,kategori_berita\n8140101,"Gadis Pujaan Dipacari Teman, Pria di Bekasi Tusuk Sesama Barber","Seorang pria berinisial EP alias A (26)tewas ditikam temannya,RA alias R (29), sesama barber di Cikarang Barat, Kabupaten Bekasi. Tersangka menusuk korban karena cemburu lantaran wanita pujaan hatinya dipacari oleh korban. ""Pelaku cemburu kepada korban karena korban berpacaran dengan saksi Sheyla (penjual es di depan barber tempat korban dan pelaku bekerja),"" kata Kapolres Metro Bekasi Kombes Mustofa, dalam keterangannya, Rabu (1/10/2025). Peristiwa berdarah itu terjadi pada Jumat (26/9) sekitar pukul 22.00 WIB. Saat itu, EP dan AR patungan membeli minuman keras sebanyak 2 botol sepulang dari bekerja dibarbershopdi Cikarang. SCROLL TO CONTINUE WITH CONTENT ""Kemudian korban dan pelaku minum bersama di kontrakan mereka di Kampung Cibitung RT 002 RW 001 Kelurahan Telagaasih, Kecamatan Cikar

In [ ]:
import os
import sys
import pandas as pd
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense

# --- Fungsi Pembersihan Teks ---
def clean_text(text):
    if not isinstance(text, str):
        return ''
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# --- Memuat Data dari File CSV ---
file_path = "detik_berita_10_pages.csv"  # sesuaikan dengan nama file hasil upload

if not os.path.exists(file_path):
    raise FileNotFoundError(f"File '{file_path}' tidak ditemukan.\n"
                            f"Upload file dulu atau cek path: {os.getcwd()}")

df = pd.read_csv(file_path)
print(f"Berhasil memuat seluruh data dari file CSV: {file_path}")

# Periksa kolom yang dibutuhkan
required_cols = ['isi_berita_diproses', 'kategori_berita']
for col in required_cols:
    if col not in df.columns:
        raise ValueError(f"Kolom '{col}' tidak ditemukan dalam file CSV.")

# --- Filter data yang valid (minimal 2 sampel per kategori) ---
df.dropna(subset=['isi_berita_diproses'], inplace=True)
counts = df['kategori_berita'].value_counts()
valid_labels = counts[counts >= 2].index
df = df[df['kategori_berita'].isin(valid_labels)]

corpus = df['isi_berita_diproses'].tolist()

# Konversi label kategori ke integer
label_to_id = {label: i for i, label in enumerate(df['kategori_berita'].unique())}
labels = [label_to_id[label] for label in df['kategori_berita']]

print("Kolom 'kategori_berita' ditemukan. Menggunakan label yang ada di file.")
print(f"Menggunakan label: {label_to_id.keys()}")

# --- Pembersihan teks ---
cleaned_corpus = [clean_text(text) for text in corpus]

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    cleaned_corpus, labels, test_size=0.2, random_state=100, stratify=labels
)

print(f"\nJumlah total data: {len(df)}")
print(f"Jumlah data latih: {len(X_train)}")
print(f"Jumlah data uji: {len(X_test)}")

# ============================================================
# 1. Klasifikasi Menggunakan TF-IDF
# ============================================================
print("\n" + "="*50)
print(">>> KLASIFIKASI MENGGUNAKAN TF-IDF")
print("="*50)

vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print("\n--- REPRESENTASI DATA DAN KONTEKS TF-IDF ---")
print("TF-IDF merepresentasikan kata sebagai vektor yang nilainya berdasarkan frekuensi (TF) dan kelangkaannya (IDF).")
print("Matriks yang dihasilkan **sparse** dan tidak menangkap makna atau konteks semantik.")
print("Contoh representasi untuk kalimat pertama (setelah preprocessing):")
contoh_tfidf = vectorizer.transform([X_train[0]])
fitur_tfidf = vectorizer.get_feature_names_out()
df_contoh_tfidf = pd.DataFrame(contoh_tfidf.T.todense(), index=fitur_tfidf, columns=['Skor TF-IDF'])
df_contoh_tfidf = df_contoh_tfidf[df_contoh_tfidf['Skor TF-IDF'] > 0].sort_values(by='Skor TF-IDF', ascending=False)
print(df_contoh_tfidf.head(50).to_string())

model_tfidf = LogisticRegression(max_iter=1000, C=0.3, class_weight='balanced')
model_tfidf.fit(X_train_tfidf, y_train)

y_pred_tfidf = model_tfidf.predict(X_test_tfidf)
print(f"\nAkurasi TF-IDF: {accuracy_score(y_test, y_pred_tfidf):.2f}")
print("Laporan Klasifikasi TF-IDF:")
print(classification_report(y_test, y_pred_tfidf, zero_division=0))
print("\n**Kegunaan Aplikasi:** TF-IDF cocok untuk tugas di mana **frekuensi kata kunci** penting, seperti klasifikasi dokumen sederhana, pencarian, atau pemfilteran spam.")

# ============================================================
# 2. Klasifikasi Menggunakan Word Embedding
# ============================================================
print("\n" + "="*50)
print(">>> KLASIFIKASI MENGGUNAKAN WORD EMBEDDING")
print("="*50)

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)
vocab_size = len(tokenizer.word_index) + 1
max_len = max(len(s.split()) for s in cleaned_corpus)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train_padded = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_len, padding='post')

embedding_dim = 100

print("\n--- REPRESENTASI DATA DAN KONTEKS WORD EMBEDDING ---")
print("Word Embedding merepresentasikan kata sebagai vektor **dense** yang belajar dari konteks.")
print("Kata-kata dengan makna serupa memiliki vektor yang dekat. Ini menangkap makna semantik.")
print(f"Setiap kata direpresentasikan sebagai vektor berdimensi {embedding_dim}.")
print("Contoh representasi dari kata 'liverpool' (vektor 100 dimensi):")

model_emb = Sequential()
model_emb.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len, name="embedding_layer"))
model_emb.add(Flatten())
model_emb.add(Dense(128, activation='relu'))
model_emb.add(Dense(len(set(labels)), activation='softmax'))

model_emb.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

model_emb.fit(X_train_padded, np.array(y_train), epochs=10, verbose=1)

embedding_layer = model_emb.get_layer('embedding_layer')
embedding_weights = embedding_layer.get_weights()[0]
word_index = tokenizer.word_index
word_to_vec = {word: embedding_weights[i] for word, i in word_index.items()}

if 'liverpool' in word_to_vec:
    print(f"Vektor untuk 'liverpool':\n{word_to_vec['liverpool'][:10]}...")
else:
    print("Kata 'liverpool' tidak ditemukan di korpus.")

loss, accuracy = model_emb.evaluate(X_test_padded, np.array(y_test), verbose=0)
print(f"\nAkurasi Word Embedding: {accuracy:.2f}")

y_pred_probs_emb = model_emb.predict(X_test_padded)
y_pred_emb = np.argmax(y_pred_probs_emb, axis=1)

print("Laporan Klasifikasi Word Embedding:")
print(classification_report(y_test, y_pred_emb, zero_division=0))
print("\n**Kegunaan Aplikasi:** Word Embedding cocok untuk tugas di mana **pemahaman makna dan konteks** diperlukan, seperti analisis sentimen, deteksi entitas bernama, atau ringkasan teks otomatis.")


Berhasil memuat seluruh data dari file CSV: detik_berita_10_pages.csv
Kolom 'kategori_berita' ditemukan. Menggunakan label yang ada di file.
Menggunakan label: dict_keys(['hukum', 'politik', 'ekonomi'])

Jumlah total data: 199
Jumlah data latih: 159
Jumlah data uji: 40

>>> KLASIFIKASI MENGGUNAKAN TF-IDF

--- REPRESENTASI DATA DAN KONTEKS TF-IDF ---
TF-IDF merepresentasikan kata sebagai vektor yang nilainya berdasarkan frekuensi (TF) dan kelangkaannya (IDF).
Matriks yang dihasilkan **sparse** dan tidak menangkap makna atau konteks semantik.
Contoh representasi untuk kalimat pertama (setelah preprocessing):
                           Skor TF-IDF
bi                            0.310574
lps                           0.296657
misbakhun                     0.267356
psk                           0.267356
independen                    0.221838
independensi                  0.197771
lembaga                       0.196870
ruu                           0.170225
anggar                        0.142

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 185ms/step - accuracy: 0.3628 - loss: 1.9489
Epoch 2/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 183ms/step - accuracy: 0.4838 - loss: 3.8495
Epoch 3/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 181ms/step - accuracy: 0.6413 - loss: 0.6636
Epoch 4/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 181ms/step - accuracy: 0.9164 - loss: 0.3815
Epoch 5/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 175ms/step - accuracy: 0.9356 - loss: 0.1975
Epoch 6/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 178ms/step - accuracy: 0.9336 - loss: 0.7291
Epoch 7/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 177ms/step - accuracy: 0.9292 - loss: 1.2625
Epoch 8/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 174ms/step - accuracy: 0.8694 - loss: 0.3341
Epoch 9/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 200ms/step - accuracy: 0.9216 - loss: 0.3203
Epoch 10/10
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 272ms/step - accuracy: 0.9601 - loss: 0.1585
Kata 'liverpool' tidak ditemukan di korpus.

Akurasi Word Embedding: 0.65
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
Laporan Klasifikasi Word Embedding:
              precisio